In [1]:
import os
import sys
os.chdir('../')
SCRIPDIR = os.path.dirname(os.path.abspath("test.ipynb"))
sys.path.append(os.path.dirname(SCRIPDIR))
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def pick_a_sample(check_df: pd.DataFrame, result_df: pd.DataFrame = None, idx:int = None):
    if check_df[check_df['ERROR_REASON'].isna()].shape[0] == 0:
        print("No error found!")
        return None
    if idx is not None:
        sample = check_df.loc[[idx]]
    else:
        sample = check_df[check_df['ERROR_REASON'].isna() & check_df['IS_ERROR'] == True].sample()
    # if result_log is not None:
    print("-----------I have the schema generated as follows:---------")
        # logs = result_log.iloc[sample.LOGS_INDEX.values[0]]
    temp_df = result_df[result_df['TESTFILE_PATH'] == sample['TESTFILE_PATH'].values[0]]
    print(sample.index)
    for sql in temp_df[(temp_df['CASE_TYPE'] == 'Statement') | (temp_df.index == sample.index.values[0])]['SQL']:
        print(sql + ';')
        # print(logs.values[0])
    print("-----------The SQL commands is:---------")
    print(sample.SQL.values[0])
    print("-----------The expected result is:---------")
    print(sample.EXPECTED_RESULT.values[0])
    print("-----------The actual result is:---------")
    print(sample.ACTUAL_RESULT.values[0])
    print("-----------The error message is:---------")
    print(sample.ERROR_MSG.values[0])

    print("-----------The test file is:---------")
    print(sample.TESTFILE_PATH.values[0])
    print("___________The index is:___________")
    print(sample.index.values[0])
    return sample.index.values[0]

# SQLite

In [3]:
dbms_name = 'sqlite'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [4]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') & (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement')& (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

0.5145124033343377
Queries: 7104, Statements: 2564
Wrong Queries: 625


In [ ]:
pick_a_sample(check_df, df)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'TYPE_CAST': lambda x: re.match(r'unrecognized token: \":\"', str(x['ERROR_MSG'])) is not None,
    'NESTF': lambda x: re.search(r'range\((.*?)\)', str(x['SQL']), re.DOTALL | re.IGNORECASE) is not None,
    "GENERAL": lambda x: re.search(r'no such function: .*', str(x['ERROR_MSG']), re.IGNORECASE) is not None,
    "EXPLAIN": lambda x: re.match(r'EXPLAIN', str(x['SQL'])) is not None,
    # "SYNTAX": lambda x: re.search(r'syntax error', str(x['ERROR_MSG']), re.IGNORECASE) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
cnt-=1

In [ ]:
sample = sample_100.iloc[cnt]

print(sample['TESTFILE_PATH'])
print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [5]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [6]:
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)

\begin{tabular}{lr}
 & DESCRIPTION \\
SUPER_TAG &  \\
Clause & 14 \\
Command & 23 \\
Function & 2 \\
Operator & 14 \\
Type & 13 \\
UFunction & 34 \\
\end{tabular}



In [ ]:
des_cnt = visible_reasons.value_counts(['SUPER_TAG','DESCRIPTION'])
print(des_cnt)
des_cnt.to_csv('output/demo_sltddb_cnt.csv')

# PostgreSQL

In [7]:
dbms_name = 'postgresql'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') & (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement')& (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

In [ ]:
pick_a_sample(check_df, df)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)
output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
cnt-=1

In [ ]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [8]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [9]:
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)

\begin{tabular}{lr}
 & DESCRIPTION \\
SUPER_TAG &  \\
Clause & 9 \\
Command & 16 \\
Function & 6 \\
IType & 3 \\
Misc & 1 \\
Setting & 3 \\
Type & 36 \\
UFunction & 26 \\
\end{tabular}



# MySQL

In [10]:
dbms_name = 'mysql'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') &
                   (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement') &
                      (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (
    df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()


In [ ]:
pick_a_sample(check_df, df)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'TYPE_CAST': lambda x: re.match(r'1064 \(42000\).*\'::', str(x['ERROR_MSG'])) is not None,
    # 'GENERAL': lambda x: re.match(r'1305 \(42000\)', str(x['ERROR_MSG'])) is not None,
    'PRAGMA': lambda x: re.match(r'1064 \(42000\).*\'PRAGMA', str(x['ERROR_MSG']), re.IGNORECASE) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)
output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
cnt-=1

In [ ]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [11]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [12]:
visible_reasons = pd.read_csv(output_reason_path)
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)

\begin{tabular}{lr}
 & DESCRIPTION \\
SUPER_TAG &  \\
Clause & 21 \\
Command & 20 \\
ICommand & 1 \\
IOperator & 1 \\
Misc & 3 \\
Operator & 10 \\
Setting & 4 \\
Type & 22 \\
UFunction & 18 \\
\end{tabular}



In [27]:
reasons = pd.read_csv("data/duckdb_suite_errors.csv")

sqlite_results = pd.read_csv("output/sqlite_duckdb_sample_100_reason.csv")
reason_df = sqlite_results[['SQL', 'EXPECTED_RESULT',
                            'ACTUAL_RESULT', 'ERROR_REASON']]
sqlite_reasons = reason_df['ERROR_REASON'].value_counts()


postgresql_results = pd.read_csv("output/postgresql_duckdb_sample_100_reason.csv")
reason_df = postgresql_results[['SQL', 'EXPECTED_RESULT',
                                'ACTUAL_RESULT', 'ERROR_REASON']]
postgresql_reasons = reason_df['ERROR_REASON'].value_counts()

# mysql
mysql_results = pd.read_csv("output/mysql_duckdb_sample_100_reason.csv")
reason_df = mysql_results[['SQL', 'EXPECTED_RESULT',
                           'ACTUAL_RESULT', 'ERROR_REASON']]
mysql_reasons = reason_df['ERROR_REASON'].value_counts()

# print(sqlite_reasons)
# print(postgresql_reasons)
# outer join two dataframes

join_df = pd.concat([sqlite_reasons, postgresql_reasons,
                    mysql_reasons], axis=1, join='outer')
# print(join_df)

join_df.fillna(0, inplace=True)
join_df = join_df.astype(int)

# pure_reasons = reasons[['TAG', 'REASON']].drop_duplicates()
pure_reasons = reasons[['TAG', 'SUPER_TAG']].drop_duplicates()
print(pure_reasons['SUPER_TAG'].value_counts())
# join the join_df and the reasons dataframe, drop duplicate rows
join_df = join_df.join(pure_reasons.set_index('TAG'),  how='left')
join_df.sort_values(by=['SUPER_TAG'], inplace=True)
detail_df = join_df.copy()

detail_df.drop(columns=['ERROR_REASON'], inplace=True)
print(detail_df.value_counts())
# sum the rows with the same SUPER_TAG
join_df.reset_index(inplace=True, drop=True)

# unique_reasons = join_df['SUPER_TAG'].drop_duplicates()
join_df = join_df.groupby('SUPER_TAG').sum()

print(join_df.to_latex())
# join_df.style.to_latex("output/error_reasons.tex", )

# unique_reasons = join_df['SUPER_TAG'].drop_duplicates()
# print(unique_reasons)

output_df = detail_df.value_counts()

schema = "\\newcommand{\\DuckDBCompatibilityIssue%s}{%s}\n"
with open("output/Variables.tex", "a") as f:
    
    f.write(schema % ('SQL', output_df['Clause'] + output_df['Command']))
    f.write(schema % ('Function', output_df['UFunction'] ))
    f.write(schema % ('Type', output_df['Type']))
    f.write(schema % ('Operator', output_df['Operator']))
    f.write(schema % ('Configuration', output_df['Setting']))
    f.write(schema % ('Semantic', output_df['Function'] + output_df['IOperator'] + output_df['ICommand'] + output_df['IType']))
    f.write(schema % ('Misc', output_df['Misc']))


Clause       18
Command      12
UFunction     9
Type          5
Misc          4
Operator      3
Function      3
IType         2
Setting       1
IOperator     1
ICommand      1
Name: SUPER_TAG, dtype: int64
SUPER_TAG
Clause       18
Command      12
UFunction     9
Type          5
Misc          4
Function      3
Operator      3
IType         2
ICommand      1
IOperator     1
Setting       1
dtype: int64
\begin{tabular}{lrrr}
\toprule
{} &  ERROR\_REASON &  ERROR\_REASON &  ERROR\_REASON \\
SUPER\_TAG &               &               &               \\
\midrule
Clause    &            14 &             9 &            21 \\
Command   &            23 &            16 &            20 \\
Function  &             2 &             6 &             0 \\
ICommand  &             0 &             0 &             1 \\
IOperator &             0 &             0 &             1 \\
IType     &             0 &             3 &             0 \\
Misc      &             0 &             1 &             3 \\
Operator 

/tmp/ipykernel_1441782/4090415852.py:47: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(join_df.to_latex())
